In [42]:
import pandas as pd
import numpy as np
import math
from collections import OrderedDict

%pylab inline
matplotlib.style.use('ggplot')
# fk_dvfs.py available 
# https://github.com/straend/System-Architecture-of-IoT/blob/master/fk_dvfs.py
%run fk_dvfs

Populating the interactive namespace from numpy and matplotlib


In [2]:
tasks = {
    'a': {'p': 4, 'Vmax': 3.3, 'Vtresh': 0.8, 'time': 0.004, 'cpu': 'PE1'},
    'b': {'p': 5, 'Vmax': 3.3, 'Vtresh': 0.8, 'time': 0.003, 'cpu': 'PE1'},
    'c': {'p': 3, 'Vmax': 2.8, 'Vtresh': 0.6, 'time': 0.007, 'cpu': 'PE2'},
    'd': {'p': 3, 'Vmax': 3.3, 'Vtresh': 0.8, 'time': 0.006, 'cpu': 'PE1'},
}
sorted_tasks = OrderedDict(sorted(tasks.items(), key=lambda t: t[0]))

# 1) Schedule - 1
Made with Excel 2016, not so much to explain about this.
![Schedule 1](C:\Users\tomas\OneDrive\note18\IoT18\Assignment2\schedule1.png)

PE1 chosen for task A for the shorter execution time
  
PE1 chosen for task B because of lower power usage
  
PE2 chosen for task C because of lower power usage
  
PE1 chosen for task D for shorter execution time

# 2) Power dissipation, energy consumption
If we assume we consume no power when no task is executed we can calculate the following


In [3]:
# first create an array of energy consumption per task
# then sum all elements in the array
total_e = sum([tasks[t]['p']*tasks[t]['time'] for t in tasks])

# calculate average power by dividing energy consumed by the time (deadline)
average_P = total_e / 0.021

print("Total energy consumption: \t%.2f mJ"%(total_e*1000))
print("Average power: \t\t\t%.2f W"%(average_P))


Total energy consumption: 	70.00 mJ
Average power: 			3.33 W


# 3) DVFS Same for all
First we calculate an e factor, then we calculate new Vdd's, times and energy consumption for every task.

In [4]:
# Calculate how much time the longest path takes, A -> C -> D
times_for_longest_path = [tasks[t]['time'] for t in ['a','c','d']]

# Deadline of 21 ms
deadline = 0.021

# Get the e factor
eF_for_all = fk_eFactor(times=times_for_longest_path, slack=deadline - sum(times_for_longest_path))

# Calculate new V0 and Vdd for both PEs
PE1_V0 = fk_V0(3.3, 0.8)
PE1_VDD = fk_Vdd(Vt = 0.8, Vmax=3.3, eFactor=eF_for_all, V0=PE1_V0)

PE2_V0 = fk_V0(2.8, 0.6)
PE2_VDD = fk_Vdd(Vt = 0.6, Vmax=2.8, eFactor=eF_for_all, V0=PE2_V0)

# loop over tasks and calculate new timing and Power dissipation for all
same_scaling = {}
for k in sorted_tasks:
    t=tasks[k]
    if t['cpu'] == 'PE1':
        vdd = PE1_VDD
    else:
        vdd = PE2_VDD
    newP = fk_PVdd(oldP=t['p'], eFactor=eF_for_all, oldV=t['Vmax'], newV=vdd)
    nt = {
        'time': t['time']*eF_for_all,
        'Vdd': vdd,
        'p': newP
    }
    same_scaling[k]=nt
    #c['time'] *= eF_for_all

# We use the same calculations as in the first part
total_e_scaled = sum([same_scaling[t]['p']*same_scaling[t]['time'] for t in same_scaling])
average_P_scaled = total_e_scaled / 0.021
print("Total energy consumption: \t%.2f mJ"%(total_e_scaled*1000))
print("Average power: \t\t\t%.2f W"%(average_P_scaled))
print("New Timings")

pd.DataFrame.from_items(
    [(t, [same_scaling[t]['time']*1000]) for t in sorted_tasks],
    orient='index', 
    columns=['time (ms)',]
)

Total energy consumption: 	54.31 mJ
Average power: 			2.59 W
New Timings


,time (ms)
a,4.941176
b,3.705882
c,8.647059
d,7.411765


![Schedule 3](C:\Users\tomas\OneDrive\note18\IoT18\Assignment2\schedule3.png)

A further improvment could be to lower the Voltage for PE1 when task B is executing to take the same time as task C

# 4) Static power dissipation

In [10]:
# Calculating for both PE's 
# create an array with values lineary distributed between 0 and 5 (Watt)(static power)
x = np.linspace(0,5, 100000)

# calculate power for different static powers(x)
# dvfs will contain a number of values for power consumption with DVFS
pe2_usage = 0

for y in tasks:
    o= tasks[y]
    e = o['p'] * o['time']
    
    
pe1_runtime = 4.94+3.71+7.41
dvfs = x * (25.8*x + 48.93+ 16.4*0.25)
gating = x * (20*x + 66 + 22*0.25)

# create the difference
diff = gating-dvfs
# find the point where the two lines crosses
for i in range(0, len(diff)):
    if diff[i]<0:
        print("At static power over **%.2f W** it makes sense to use power gating"%x[i])
        break

At static power over **3.18 W** it makes sense to use power gating


In [38]:
# Calculating for both PE's 
# create an array with values lineary distributed between 0 and 5 (Watt)(static power)
x = np.linspace(0,5, 100000)

# calculate power for different static powers(x)

totaltime = (4+3+4+6+4)/1000
totaltime_dvfs = (4.94+3.71+4.94+7.41)/1000

pe1_tasks = [tasks[x] for x in tasks if tasks[x]['cpu'] == 'PE1']
pe2_tasks = [tasks[x] for x in tasks if tasks[x]['cpu'] == 'PE2']

# Usages for DVFS
pe1_active = sum([x['time'] for x in pe1_tasks])
pe1_usage = sum([x['time']*x['p'] for x in pe1_tasks])
pe1_static_usage = (totaltime-pe1_active) * x
pe1_total = pe1_static_usage+pe1_usage

pe2_active = sum([x['time'] for x in pe2_tasks])
pe2_usage = sum([x['time']*x['p'] for x in pe2_tasks])
pe2_static_usage = (totaltime-pe2_active) * x
pe2_total = pe2_static_usage+pe2_usage


# Usage for Sleeping
sleep_dissipate = 0.25

pe1_sleep = (totaltime-pe1_active) * sleep_dissipate
pe2_sleep = (totaltime-pe2_active) * sleep_dissipate

# Loop over values and show result
for i in range(len(pe1_static_usage)-1, -len(pe1_static_usage), -1):
    if pe1_static_usage[i] > pe1_sleep:
        print("PE1 Power Gating is prefered when static usage is more than {}W".format(pe1_static_usage[i]))
        break
for i in range(len(pe2_static_usage)-1, -len(pe2_static_usage), -1):
    if pe2_static_usage[i] > pe2_sleep:
        print("PE2 Power Gating is prefered when static usage is more than {}W".format(pe1_static_usage[i]))
        break


PE1 Power Gating is prefered when static usage is more than 0.04W
PE2 Power Gating is prefered when static usage is more than 0.04W
